# salesForecasting
- Sara Echeverría 21371
- Ricardo Mendez 21289
- Melissa Pérez 21385

Repository link: https://github.com/bl33h/salesForecasting

## Task 1: Práctica

In [1]:
# librerías
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [2]:
# configuración para uso de GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

### 1. Preparación de datos:

In [3]:
# carga de datos
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [4]:
# inspección inicial de los datos
print("Train DataFrame shape:", train_df.shape)
print("Test DataFrame shape:", test_df.shape)

train_df.head()

Train DataFrame shape: (913000, 4)
Test DataFrame shape: (45000, 4)


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [5]:
# valores faltantes en cada dataset
print("Valores faltantes en Train:")
print(train_df.isnull().sum())

print("\nValores faltantes en Test:")
print(test_df.isnull().sum())

Valores faltantes en Train:
date     0
store    0
item     0
sales    0
dtype: int64

Valores faltantes en Test:
id       0
date     0
store    0
item     0
dtype: int64


In [6]:
scaler = MinMaxScaler()
train_df['sales_scaled'] = scaler.fit_transform(train_df[['sales']])

# Display the first few rows of the train dataset after normalization
df_prepared = train_df.head()
print(df_prepared)

         date  store  item  sales  sales_scaled
0  2013-01-01      1     1     13      0.056277
1  2013-01-02      1     1     11      0.047619
2  2013-01-03      1     1     14      0.060606
3  2013-01-04      1     1     13      0.056277
4  2013-01-05      1     1     10      0.043290


### 2. Preprocesamiento de datos

In [7]:
train_df = train_df.sort_values(by='date')

# 90 días como 3 meses
test_size = 90 * len(train_df['store'].unique()) * len(train_df['item'].unique())

# división de entrenamiento, validación y prueba
train_data = train_df.iloc[:-test_size]
test_data = train_df.iloc[-test_size:]

# tamaño del conjunto de validación como el 10% del conjunto de entrenamiento
val_size = int(0.1 * len(train_data))
val_data = train_data.iloc[-val_size:]
train_data = train_data.iloc[:-val_size]

print("Training set size:", train_data.shape)
print("Validation set size:", val_data.shape)
print("Test set size:", test_data.shape)

Training set size: (781200, 5)
Validation set size: (86800, 5)
Test set size: (45000, 5)


In [8]:
# crear secuencias de ventas históricas y objetivos futuros
def create_sequences(data, window_size=30, forecast_size=90):
    X, y = [], []
    for i in range(len(data) - window_size - forecast_size):
        X.append(data.iloc[i:i + window_size]['sales_scaled'].values)
        y.append(data.iloc[i + window_size:i + window_size + forecast_size]['sales_scaled'].values)
    return np.array(X), np.array(y)

# generación de secuencias
window_size = 30
forecast_size = 90

X_train, y_train = create_sequences(train_data, window_size, forecast_size)
X_val, y_val = create_sequences(val_data, window_size, forecast_size)
X_test, y_test = create_sequences(test_data, window_size, forecast_size)

print("Training sequences:", X_train.shape, y_train.shape)
print("Validation sequences:", X_val.shape, y_val.shape)
print("Test sequences:", X_test.shape, y_test.shape)

Training sequences: (781080, 30) (781080, 90)
Validation sequences: (86680, 30) (86680, 90)
Test sequences: (44880, 30) (44880, 90)


### 3. Selección de modelo: LSTM
### 4. Arquitectura del modelo

In [9]:
# input shape
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

def create_lstm_model(input_shape):
    model = Sequential()

    # primera capa
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # segunda capa
    model.add(LSTM(100, activation='relu', return_sequences=False))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # capa Densa final para predicción
    model.add(Dense(forecast_size, activation='relu'))

    # compilar el modelo
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [10]:
# crear el modelo
input_shape = (X_train.shape[1], X_train.shape[2])
model = create_lstm_model(input_shape)

model.summary()

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 100)        │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 100)        │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 90)             │         9,090 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,090 (512.07 KB)

 Trainable params: 130,690 (510.51 KB)

 Non-trainable params: 400 (1.56 KB)

### 5. Entrenamiento del modelo

In [ ]:
# early stopping para detener el entrenamiento si el modelo no mejora
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# entrenar el modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/20


In [ ]:
# grafica de la pérdida durante el entrenamiento
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss During Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### 6. Evaluación del modelo:

### 7. Ajuste de hiperparámetros:

### 8. Forecasting:

### 9. Visualización:

### 10. Interpretabilidad del modelo:

## Task 2: Teoría

### 1. ¿Cuál es el problema del gradiente de fuga en las redes LSTM y cómo afecta la efectividad de LSTM para el pronóstico de series temporales?
En este contexto, al gradiente utilizado en las LSTM dificulta aprender dependencias a largo plazo. Su arquitectura ayudn a retener información relevante, mejorando el pronóstico de series temporales.

### 2. ¿Cómo se aborda la estacionalidad en los datos de series temporales cuando se utilizan LSTM para realizar pronósticos y qué papel juega la diferenciación en el proceso?
Esta se maneja por medio de la diferenciación para eliminar patrones estacionales, permitiendo un pronóstico más efectivo al enfocar las dependencias temporales significativas.

### 3. ¿Cuál es el concepto de "tamaño de ventana" en el pronóstico de series temporales con LSTM y cómo afecta la elección del tamaño de ventana a la capacidad del modelo para capturar patrones a corto y largo plazo?
Este hace referencia a la cantidad de datos pasados que usa el LSTM para predecir futuros valores. Ventanas más grandes capturan patrones a largo plazo, mientras que ventanas más pequeñas se enfocan en patrones recientes. En ambos casos puede haber pérdida de información.